# 2. The current Jupyter Notebook will cover the Second Phase of the project "RAG process"

According to the article "What is retrieval augmented generation (RAG)?" by IBM, RAG is "an architecture for optimizing the performance of an artificial intelligence (AI) model by connecting it with external knowledge bases.".

The mechanics of this architecture are detailed in the seminal paper "Retrieval-augmented generation for knowledge-intensive NLP tasks" (Lewis et al., 2020), which demonstrates that it is not necessary to retrain a massive AI model to teach it new facts.

## 2.1 Develop the retrieval engine to identify narrative episodes based on vector similarity

For this part of the project we will use Ollama to be the server of our RAG system so it will allow us to run LLM locally, therefore python will work as the "client".

The current code will do the search based on the query and send it to Ollama to generate the answer.

In [ ]:
# Import the needed libraries

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

# Define the configuration for the needed to use Ollama

VECTOR_DATABASE_PATH = r"C:\Users\lonel\OneDrive\Escritorio\Re Zero NLP Project\vector_database"
MODEL_NAME = "llama3.1:8b"
print("Configuration to use Ollama correctly created!")

# Initialize the Embedding Model

embedding_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
print("Embedding Model correctly created!")

# Connect the embedding model to the vector database

vector_database = Chroma(persist_directory=VECTOR_DATABASE_PATH,
                        embedding_function=embedding_model)
print("Embedding Model correctly connected to the database!")

# Connect to Ollama

llm = ChatOllama(model=MODEL_NAME)

# Create a function that performs the vector similarity search

def query_question(question):
    """The current function takes as the input a question in the Terminal"""

    print("Searching in the vector database for the question {}".format(question))

    # Perform the search (Retrieve)

    retrieves = vector_database.similarity_search(question, k=3) # K is the number of chunks the model will retrieve to answer the question.

    if not retrieves:
        print("No relevant information found with the current data stored in my memory. Try adding more data.")
        return
    
    # Print the chunks the search found after the question

    print("\nRetrieved Chunks:")
    for i, retrieve in enumerate(retrieves):
        print(f"Chunk {i+1} extracted from {retrieve.metadata.get('source', 'unknown')}")
        print(f"and contains the text:")
        print(f"{retrieve.page_content}")
    
    # Combine the retrieved text into one context block

    context_text = "\n\n---\n\n".join([retrieve.page_content for retrieve in retrieves])

    # Define the prompt to guide the answer (Augment)

    prompt = f"""
    You are a helpful assistant for the novel Re:Zero. 
    Use the following pieces of retrieved context to answer the user's question.
    If the answer is not in the context, just say that you don't know.
    
    Context:
    {context_text}
    
    Question:
    {question}
    
    Answer:
    """

    # Ask to the LLM (Generate)

    print("The LLM has been asked the question and is now generating the answer!")

    response = llm.invoke(prompt)

    # Print the result

    print("\n" + "="*30)
    print(f"Question: {question}")
    print(f"Answer: {response.content}")
    print("="*30)

Configuration to use Ollama correctly created!
Embedding Model correctly created!
Embedding Model correctly connected to the database!


In [5]:
# Test the RAG system

if __name__ == "__main__":
    while True:
        user_input = input("\nAsk about Arc 1 (or type 'exit'): ")
        if user_input.lower() == 'exit':
            break
        query_question(user_input) # I way to solve the CUDA error is to copy and paste the next line in your CMD: set OLLAMA_USE_CPU=true

Searching in the vector database for the question Who killed Rom?

Retrieved Chunks:
Chunk 1 extracted from arc-1-chapter-9.txt
and have the text Rom’s face was stern as he answered Subaru’s tactless question.

He then brought the bottle he had been pouring out of to his mouth, and as he drank,

“Because of this, most of us were wiped out. Even in the capital, I haven’t seen any other giants.”

“Yer strong even without eatin’, sho kewl. … Gunna throw up.”

“I’m saying something sad here and you respond like that?”

He wasn’t about to let someone’s sob story kill his mood.

As Subaru blocked his ears and interrupted the story, Rom gave up on telling it and started eating his beans.

The two of them passed their time silently eating those terrible beans as a side to their alcohol.

Eventually there was a coded knock on the door, by which time the sun had already set for the most part.

Subaru who had been nodding off raised his head, and Rom nimbly approached the door in response to the 